In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
 #   for filename in filenames:
  #      print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np
import pandas as pd
import keras
import cv2
from matplotlib import pyplot as plt
import os
import random
from PIL import Image


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

sample_path = r"/kaggle/input/landmark-recognition-2020/sample_submission.csv"
train_path = r"/kaggle/input/landmark-recognition-2020/train.csv"
base_path = r"/kaggle/input/landmark-recognition-2020/train"
test_path = r"/kaggle/input/landmark-recognition-2020/test"


df = pd.read_csv("../input/landmark-recognition-2020/train.csv")# Read the CSV file containing the training labels etc.

df = df.loc[:15000,:]




In [ ]:
print("Size of training data:", df.shape)
#Count how many unique landmarks there are, that is to say the amount of classes
print("Number of unique classes:", num_classes)

In [ ]:
data = pd.DataFrame(df['landmark_id'].value_counts()) #make data frame that is easier to use
#index the data frame
data.reset_index(inplace=True) 
data.columns=['landmark_id','count']


data = data.iloc[80:200]
num_classes = len(data["landmark_id"].unique())




In [ ]:
# plt.hist(data['count'],100,range = (0,944),label = 'test')#Histogram of the distribution
# plt.xlabel("Amount of images")
# plt.ylabel("Occurences")

In [ ]:
# print("Amount of classes with five and less datapoints:", (data['count'].between(0,5)).sum()) 

# print("Amount of classes with with between five and 10 datapoints:", (data['count'].between(5,10)).sum())

# n = plt.hist(df["landmark_id"],bins=df["landmark_id"].unique())
# freq_info = n[0]

# plt.xlim(0,data['landmark_id'].max())
# plt.ylim(0,data['count'].max())
# plt.xlabel('Landmark ID')
# plt.ylabel('Number of images')

In [ ]:
from sklearn.preprocessing import LabelEncoder
lencoder = LabelEncoder()
lencoder.fit(data["landmark_id"])

def encode_label(lbl):
    return lencoder.transform(lbl)
    
def decode_label(lbl):
    return lencoder.inverse_transform(lbl)

In [ ]:
def get_image_from_number(num):
    fname, label = df.loc[num,:]
    fname = fname + ".jpg"
    f1 = fname[0]
    f2 = fname[1]
    f3 = fname[2]
    path = os.path.join(f1,f2,f3,fname)
    im = cv2.imread(os.path.join(base_path,path))
    return im, label

### Function used for processing the data, fitted into a data generator.
def get_image_from_number(num, df):
    fname, label = df.iloc[num,:]
    fname = fname + ".jpg"
    f1 = fname[0]
    f2 = fname[1]
    f3 = fname[2]
    path = os.path.join(f1,f2,f3,fname)
    im = cv2.imread(os.path.join(base_path,path))
    return im, label

In [ ]:
from keras.applications import VGG19
from keras.layers import *
from keras import Sequential

source_model = VGG19(weights=None)
#new_layer = Dense(num_classes, activation=activations.softmax, name='prediction')
drop_layer = Dropout(0.5)
drop_layer2 = Dropout(0.5)

In [ ]:
model = Sequential()
for layer in source_model.layers[:-1]: # go through until last layer
    if layer == source_model.layers[-25]:
        model.add(BatchNormalization())
    model.add(layer)
    #if layer == source_model.layers[-3]:
     #   model.add(drop_layer)
    #model.add(drop_layer2)
model.add(Dense(num_classes, activation="softmax"))
model.summary()


opt1 = keras.optimizers.RMSprop(learning_rate = 0.0001, momentum = 0.09)
opt2 = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07)
model.compile(optimizer=opt1,
             loss="sparse_categorical_crossentropy",
             metrics=["accuracy"])

#sgd = SGD(lr=learning_rate, decay=decay_speed, momentum=momentum, nesterov=True)
# rms = keras.optimizers.RMSprop(lr=learning_rate, momentum=momentum)
# model.compile(optimizer=rms,
#               loss=loss_function,
#               metrics=["accuracy"])
# print("Model compiled! \n")

In [ ]:
def image_reshape(im, target_size):
    return cv2.resize(im, target_size)
    
def get_batch(dataframe,start, batch_size):
    image_array = []
    label_array = []
    
    end_img = start+batch_size
    if end_img > len(dataframe):
        end_img = len(dataframe)

    for idx in range(start, end_img):
        n = idx
        im, label = get_image_from_number(n, dataframe)
        im = image_reshape(im, (224, 224)) / 255.0
        image_array.append(im)
        label_array.append(label)
        
    label_array = encode_label(label_array)
    return np.array(image_array), np.array(label_array)

In [ ]:
from keras.utils.np_utils import to_categorical
name = []
cat = []
lsit  = list(data['landmark_id'])

for i in range(len(df)):
    if df.iloc[i,1] not in lsit:
        #print("YES")
        continue
        
    else:
        name.append(df.iloc[i,0])
        cat.append(df.iloc[i,1])
dataF=pd.DataFrame(list(zip(name, cat)),columns=['FILE','CLASS'])




In [ ]:
del name
del cat
del data
del df
del lsit

In [ ]:
print(num_classes)

In [ ]:
#dataF['CLASS'] =  to_categorical(dataF['CLASS'], num_classes=num_classes)
print(dataF.shape)

In [ ]:
import sys
batch_size = 200
epoch_shuffle = True
weight_classes = True
epochs = 50

# Split train data up into 80% and 20% validation


train,validate = np.split(dataF.sample(frac=1), [int(.8*len(dataF))])
print("Training on:", len(train), "samples")
print("Validation on:", len(validate), "samples")

    
for e in range(epochs):
    print("Epoch: ", str(e+1) + "/" + str(epochs))
    if epoch_shuffle:
        train = train.sample(frac = 1)
    for it in range(int(np.ceil(len(train)/batch_size))):
        #print("Current batch number:",it)
        X_train, y_train = get_batch(train, it*batch_size, batch_size)
        training_datagen = ImageDataGenerator(rotation_range=40, 
                        width_shift_range=0.2, 
                        height_shift_range=0.2, 
                        zoom_range=0.2, 
                        horizontal_flip=True, 
                        vertical_flip=True,
                        shear_range=0.2) 

        train_generator = training_datagen.flow(X_train, y_train)
        training_datagen.fit(X_train)
        del X_train
        del y_train
        history = model.fit_generator(train_generator)
        #model.train_on_batch(X_train, y_train)
        
model.save("Model.h5")

In [ ]:
### Test on training set
batch_size = 20

errors = 0
good_preds = []
bad_preds = []


for it in range(int(np.ceil(len(validate)/batch_size))):

    
    X_train, y_train = get_batch(validate, it*batch_size, batch_size)
    
    result = model.predict(X_train)
    cla = np.argmax(result, axis=1)
    for idx, res in enumerate(result):
        print("Class:", cla[idx], "- Confidence:", np.round(res[cla[idx]],2), "- GT:", y_train[idx])
        if cla[idx] != y_train[idx]:
            errors = errors + 1
            bad_preds.append([batch_size*it + idx, cla[idx], res[cla[idx]]])
        else:
            good_preds.append([batch_size*it + idx, cla[idx], res[cla[idx]]])

print("Errors: ", errors, "Acc:", np.round(100*(len(validate)-errors)/len(validate),2))

In [ ]:
from sklearn.metrics import precision_recall_fscore_support as score
precision, recall, fscore, support = score(y_train, result)

print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support))

In [ ]:
good_preds = np.array(good_preds)
good_preds = np.array(sorted(good_preds, key = lambda x: x[2], reverse=True))

print("5 images where classification went well:")
fig=plt.figure(figsize=(16, 16))
for i in range(2,6):
    n = int(good_preds[i,0])
    img, lbl = get_image_from_number(n, validate)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    fig.add_subplot(1,6, i)
    plt.imshow(img)
    lbl2 = np.array(int(good_preds[i,1])).reshape(1,1)
    sample_cn = list(dataF['CLASS']).count(lbl)
    plt.title("Label: " + str(lbl) + "\nClassified as: " + str(decode_label(lbl2)) + "\nSamples in class " + str(lbl) + ": " + str(sample_cn))
    plt.axis('off')
plt.show()

In [ ]:
bad_preds = np.array(bad_preds)
bad_preds = np.array(sorted(bad_preds, key = lambda x: x[2], reverse=True))

print("5 images where classification failed:")
fig=plt.figure(figsize=(16, 16))
for i in range(1,6):
    n = int(bad_preds[i,0])
    img, lbl = get_image_from_number(n, validate)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    fig.add_subplot(1, 5, i)
    plt.imshow(img)
    lbl2 = np.array(int(good_preds[i,1])).reshape(1,1)
    sample_cn = list(dataF['CLASS']).count(lbl)
    plt.title("Label: " + str(lbl) + "\nClassified as: " + str(decode_label(lbl2)) + "\nSamples in class " + str(lbl) + ": " + str(sample_cn))
    plt.axis('off')
    
plt.show()

In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import os
from urllib.request import urlopen,urlretrieve
from PIL import Image
from tqdm import tqdm_notebook
%matplotlib inline
from sklearn.utils import shuffle
import cv2
#import tensorflow
#from resnets_utils import *

from keras.models import load_model
from sklearn.datasets import load_files   
from keras.utils import np_utils
from glob import glob
from keras import applications
from keras.preprocessing.image import ImageDataGenerator 
from keras import optimizers
from keras.models import Sequential,Model,load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D,GlobalAveragePooling2D
from keras.callbacks import TensorBoard,ReduceLROnPlateau,ModelCheckpoint

In [ ]:
train,validate = np.split(dataF.sample(frac=1), [int(.7*len(dataF))])
print("Training on:", len(train), "samples")
print("Validation on:", len(validate), "samples")

In [ ]:
img_height,img_width = 224,224

#If imagenet weights are being loaded, 
#input must have a static square shape (one of (128, 128), (160, 160), (192, 192), or (224, 224))
base_model = applications.resnet50.ResNet50(weights= None, include_top=False, input_shape= (img_height,img_width,3))

In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.7)(x)
predictions = Dense(num_classes, activation= 'softmax')(x)
model = Model(inputs = base_model.input, outputs = predictions)

In [ ]:
from keras.optimizers import SGD, Adam
# sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
adam = Adam(lr=0.0001)
model.compile(optimizer= adam, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:

del X_test
del Y_test


In [ ]:
X_train, Y_train = get_batch(train, 0, len(train))
del(train)

In [ ]:
model.fit(X_train, Y_train, epochs = 60, batch_size = 48)

In [ ]:
X_test, Y_test = get_batch(validate,0, len(validate)-200)

In [ ]:
preds = model.evaluate(X_test, Y_test)

print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

In [ ]:
t = model.predict(X_test)
print(t)

In [ ]:
batch_size = 16

errors = 0
good_preds = []
bad_preds = []
cla = np.argmax(t,axis=1)
for idx, res in enumerate(t):
        print("Class:", cla[idx], "- Confidence:", np.round(res[cla[idx]],2), "- GT:", Y_test[idx])
        if cla[idx] != Y_test[idx]:
            errors = errors + 1
            bad_preds.append([idx, cla[idx], res[cla[idx]]])
        else:
            good_preds.append([idx, cla[idx], res[cla[idx]]])

print("Errors: ", errors, "Acc:", np.round(100*(len(validate)-errors)/len(validate),2))

In [ ]:
from keras.layers import  Flatten, Dense, Dropout
from keras.applications import VGG16
from keras.models import Model
from keras import optimizers
from keras.optimizers import Adam
from keras.layers import Dense, GlobalAveragePooling2D
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization, AveragePooling2D, GlobalAveragePooling2D
vgg16_model = VGG16(weights = 'imagenet', include_top = False,input_shape=(224,224,3))
x = vgg16_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

predictions = Dense(num_classes, activation = 'softmax')(x)
model2 = Model(vgg16_model.input,predictions)
for layer in vgg16_model.layers:
    layer.trainable = False
optimizer = Adam(lr=0.0002)
model2.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

model2.summary()

In [ ]:
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

In [ ]:
training_datagen = ImageDataGenerator(rotation_range=40, 
                        width_shift_range=0.2, 
                        height_shift_range=0.2, 
                        zoom_range=0.2, 
                        horizontal_flip=True, 
                        vertical_flip=True,
                        shear_range=0.2) 

train_generator = training_datagen.flow(X_train, Y_train,batch_size=64)
training_datagen.fit(X_train)
del X_train
del Y_train
from keras import callbacks



In [ ]:
filepath="Best1.hdf5"
checkpoint = callbacks.ModelCheckpoint(filepath, monitor='val_loss',save_best_only=True, mode='min',verbose=1)
callbacks_list = [checkpoint]

history = model2.fit_generator(train_generator, steps_per_epoch=35, epochs=100,
                              validation_data=(X_test, Y_test),validation_steps=50,callbacks=callbacks_list)

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(acc))
plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()
plt.show()

In [ ]:
model2.load_weights("Best1.hdf5")
score = model2.evaluate(X_test, Y_test ,verbose=1)
print('Test Loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
#200 epochs
history = model2.fit_generator(train_generator, steps_per_epoch=35, epochs=80,
                              validation_data=(X_test, Y_test),validation_steps=50,callbacks=callbacks_list)